Домашнее задание 1

Срок выполнения 30.09.2024

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок (можно вывести топ 10)

Файл можете взять из прошлого домашнего задания + сохраните результат на диск

In [ ]:
!pip install pyspark findspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').set('spark.serializer', 'org.apache.spark.serializer.KryoSerializer')\
                  .set('spark.dynamicAllocation.enabled', 'true')\
                  .set('spark.shuffle.service.enabled', 'true') #трекер, чтобы возвращать ресурсы
sc = SparkContext(conf=conf)
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
user_ratedmovies_rdd = sc.textFile('user_ratedmovies.dat')

user_ratedmovies_rdd = sc.parallelize(user_ratedmovies_rdd.collect()[1:], numSlices=2)

In [ ]:
def mapper_movies(text):
    film_id = int(text.split('\t')[1])
    rating_more_then_4 = 1 if float(text.split('\t')[2]) > 4 else 0
    films_with_rating = (film_id, rating_more_then_4)
    return films_with_rating

In [ ]:
result = user_ratedmovies_rdd.map(mapper_movies)\
                    .reduceByKey(lambda x, y: x + y)\
                    .sortBy(lambda x: x[1], ascending=False)\
                    .take(10)

result

[(2571, 900),
 (318, 887),
 (296, 878),
 (2959, 828),
 (4993, 756),
 (7153, 719),
 (5952, 697),
 (858, 690),
 (50, 688),
 (2858, 680)]

In [ ]:
with open('films_result.txt', 'w') as f:
  for film_id, count in result:
    f.write(f'{film_id}\t{count}\n')

**Домашнее задание 2**

В этом задании у вас есть файл с обученным word2vec на произведении Достоевского Преступление и наказание. Файл - list, каждый элемент которого слово и его вектор в формате ('word', [vector]). Необходимо для каждого слова собрать список его top 10 похожих слов по косинусной метрике
Результат также сохраните на диск и выведите синонимы для слова 'топор' и 'деньга'.
Файл в пикле, так что для считывания воспользуйтесь не spark, можете взять любимый pandas

Файл не очень уж и маленький, рекомендую сначала написать код на кошках/собачках, а потом уже на всем, так как ядра 2, считаться будет очень долго.
В качестве одного из вариантов можете рассмотреть метод cartesian

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
w2v_list = pd.read_pickle('w2v_vectors.pickle')

w2v_spark = sc.parallelize(w2v_list)

In [ ]:
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
def lists_find_top_10_similar(word_vector, words_vectors):
    similarities = []
    for row in words_vectors:
        similarities.append((row[0], cosine_similarity(word_vector, row[1])))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[1:11]

In [ ]:
w2v_spark_list = w2v_spark.collect()

result = w2v_spark.map(lambda row: (row[0], lists_find_top_10_similar(row[1], w2v_spark_list)))

In [ ]:
for word in ['топор', 'деньга']:
    similar_words = result.filter(lambda row: row[0] == word).collect()[0][1]
    print(f"Топ-10 похожих слов для '{word}':")
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity}")
    print()

Топ-10 похожих слов для 'топор':
  левый: 0.9236524701118469
  старуха: 0.922432005405426
  колено: 0.920665979385376
  побежать: 0.912059485912323
  ломать: 0.910239040851593
  бросить: 0.9063428640365601
  схватить: 0.9024109840393066
  обнять: 0.8999080657958984
  бить: 0.898546040058136
  положить: 0.8972360491752625

Топ-10 похожих слов для 'деньга':
  вещь: 0.9063147902488708
  отдать: 0.8706483244895935
  убить: 0.8420270085334778
  значит: 0.838628351688385
  известно: 0.8266707062721252
  отец: 0.8146180510520935
  получить: 0.8137906193733215
  ограбить: 0.8044242858886719
  копейка: 0.8012077808380127
  полечка: 0.7804851531982422



In [ ]:
result.coalesce(1).saveAsPickleFile('result_1.pickle')

Версия 2 с использованием cartesian


In [ ]:
def mapper_similarities(row):
    return (row[0][0], (row[1][0], cosine_similarity(row[0][1], row[1][1])))

In [ ]:
def mapper_sort(row):
    return sorted(row, key=lambda x: x[1], reverse=True)[1:11]

In [ ]:
result_2 = w2v_spark.cartesian(w2v_spark)\
                    .map(mapper_similarities)\
                    .groupByKey()\
                    .mapValues(mapper_sort)

In [ ]:
for word in ['топор', 'деньга']:
    similar_words = result_2.filter(lambda row: row[0] == word).collect()[0][1]
    print(f"Топ-10 похожих слов для '{word}':")
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity}")
    print()

Топ-10 похожих слов для 'топор':
  левый: 0.9236524701118469
  старуха: 0.922432005405426
  колено: 0.920665979385376
  побежать: 0.912059485912323
  ломать: 0.910239040851593
  бросить: 0.9063428640365601
  схватить: 0.9024109840393066
  обнять: 0.8999080657958984
  бить: 0.898546040058136
  положить: 0.8972360491752625

Топ-10 похожих слов для 'деньга':
  вещь: 0.9063147902488708
  отдать: 0.8706483244895935
  убить: 0.8420270085334778
  значит: 0.838628351688385
  известно: 0.8266707062721252
  отец: 0.8146180510520935
  получить: 0.8137906193733215
  ограбить: 0.8044242858886719
  копейка: 0.8012077808380127
  полечка: 0.7804851531982422



In [ ]:
result_2.coalesce(1).saveAsPickleFile('result_2.pickle')